In [44]:
!pip install folium geocoder

In [45]:
import folium, pandas as pd, requests, numpy as np, geocoder, json
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

# Using Pandas:
1. read_html to parse the table into a dataframe
2. replace "Not assigned" values in "Boroughs" with NaNs, before dropping them.

### No duplicate postal codes were found. All postal codes are unique, seemingly already grouped.

In [46]:
dfs = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = dfs[0].replace("Not assigned", np.nan).dropna(subset=['Borough'])

print(df['Postal Code'].value_counts().max())
print(df.head(10))

df.shape

1
   Postal Code           Borough                                Neighbourhood
2          M3A        North York                                    Parkwoods
3          M4A        North York                             Victoria Village
4          M5A  Downtown Toronto                    Regent Park, Harbourfront
5          M6A        North York             Lawrence Manor, Lawrence Heights
6          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
8          M9A         Etobicoke      Islington Avenue, Humber Valley Village
9          M1B       Scarborough                               Malvern, Rouge
11         M3B        North York                                    Don Mills
12         M4B         East York              Parkview Hill, Woodbine Gardens
13         M5B  Downtown Toronto                     Garden District, Ryerson


(103, 3)

# Using BeautifulSoup:

1. Make a get request with requests.
2. Load it on BS and parse the table
3. Load data on a DataFrame

In [47]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(response.text, "html.parser")
table = soup.table

In [48]:
data = []

for row in table.findAll('tr'):
     
    row_list = [i for i in row.text.split('\n') if i != '']
    data.append(row_list)
    
df = pd.DataFrame(data[1:], columns=data[0])
df.replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)

print(df.head(10))
df.shape

   Postal Code           Borough                                Neighbourhood
2          M3A        North York                                    Parkwoods
3          M4A        North York                             Victoria Village
4          M5A  Downtown Toronto                    Regent Park, Harbourfront
5          M6A        North York             Lawrence Manor, Lawrence Heights
6          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
8          M9A         Etobicoke      Islington Avenue, Humber Valley Village
9          M1B       Scarborough                               Malvern, Rouge
11         M3B        North York                                    Don Mills
12         M4B         East York              Parkview Hill, Woodbine Gardens
13         M5B  Downtown Toronto                     Garden District, Ryerson


(103, 3)

In [49]:
with requests.Session() as session:
    
    def get_lat_lng(x, session=session):
        g = geocoder.arcgis(', '.join(x)).json
        return g['lat'], g['lng']
    
    df['Neighbourhood Latitude'], df['Neighbourhood Longitude'] = zip(*df.aggregate(get_lat_lng, axis=1))
    
df.reset_index(inplace=True, drop=True)
df

,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude
0,M3A,North York,Parkwoods,43.752450,-79.329910
1,M4A,North York,Victoria Village,43.730570,-79.313060
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655120,-79.362640
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723570,-79.437110
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.660673,-79.390835
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.648527,-79.517933
99,M4Y,Downtown Toronto,Church and Wellesley,43.665700,-79.380930
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",40.473733,-80.732340
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632780,-79.489450


In [50]:
# The code was removed by Watson Studio for sharing.

Limit boroughs to the ones containing the word "Toronto":

In [51]:
toronto_neighbourhoods = df[df.Borough.str.contains('Toronto')].reset_index(drop=True)
toronto_neighbourhoods.shape

(39, 5)

In [52]:
def explore_venues(dataframe):
    
    list_of_venues = []
    
    for index, row in dataframe.iterrows():
        
        Neighbourhood_lat = row['Neighbourhood Latitude']
        Neighbourhood_lng = row['Neighbourhood Longitude']

        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&auth_token={}&v={}&ll={},{}&radius={}&limit={}".format(
                    Client_Id,
                    Client_Secret,
                    Auth_Token,
                    Version,
                    Neighbourhood_lat,
                    Neighbourhood_lng,
                    500,
                    Limit
                )
        
        r = requests.get(url)
        print("Remaining API calls:", r.headers['X-RateLimit-Remaining'])
        
        if r.status_code >= 400:
            
            print('Bad response')
            break
        
        data = r.json()['response']['groups'][0]['items']
        print('Returned venues: ', len(data))
        
        list_of_venues.append([
            
                (
                row['Postal Code'],
                row['Borough'],
                row['Neighbourhood'],
                row['Neighbourhood Latitude'],
                row['Neighbourhood Longitude'],
                venue['venue']['name'],
                venue['venue']['location']['lat'],
                venue['venue']['location']['lng'],
                venue['venue']['categories'][0]['name']
                ) for venue in data]
            )
        
    frame = pd.DataFrame([item for neighborhood in list_of_venues for item in neighborhood],
                        columns=[*dataframe.columns, 'Venue Name', 'Venue Latitude', 'Venue Longitude', 'Venue Category'])
    
    return frame
        

toronto_venues = explore_venues(toronto_neighbourhoods)

Remaining API calls: 4921
Returned venues:  20
Remaining API calls: 4920
Returned venues:  56
Remaining API calls: 4919
Returned venues:  100
Remaining API calls: 4918
Returned venues:  26
Remaining API calls: 4917
Returned venues:  23
Remaining API calls: 4916
Returned venues:  100
Remaining API calls: 4915
Returned venues:  87
Remaining API calls: 4914
Returned venues:  15
Remaining API calls: 4913
Returned venues:  100
Remaining API calls: 4912
Returned venues:  44
Remaining API calls: 4911
Returned venues:  58
Remaining API calls: 4910
Returned venues:  37
Remaining API calls: 4909
Returned venues:  0
Remaining API calls: 4908
Returned venues:  100
Remaining API calls: 4907
Returned venues:  84
Remaining API calls: 4906
Returned venues:  23
Remaining API calls: 4905
Returned venues:  100
Remaining API calls: 4904
Returned venues:  0
Remaining API calls: 4903
Returned venues:  26
Remaining API calls: 4902
Returned venues:  4
Remaining API calls: 4901
Returned venues:  2
Remaining AP

*^ 3 Neighbourhoods returned no venues, which need to be dropped in the end.*

In [53]:
print(toronto_venues.shape)
toronto_venues

(1770, 9)


,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Berkeley Church,43.655123,-79.365873,Event Space
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
...,...,...,...,...,...,...,...,...,...
1765,M4Y,Downtown Toronto,Church and Wellesley,43.66570,-79.38093,Tim Hortons,43.669913,-79.379337,Coffee Shop
1766,M4Y,Downtown Toronto,Church and Wellesley,43.66570,-79.38093,Rogers Radio Cafe (2nd Floor),43.669859,-79.379633,Café
1767,M4Y,Downtown Toronto,Church and Wellesley,43.66570,-79.38093,Subway,43.666814,-79.376300,Sandwich Place
1768,M4Y,Downtown Toronto,Church and Wellesley,43.66570,-79.38093,Rogers Cafeteria,43.669904,-79.379748,Food Court


Extract neighbourhood and venue category columns on a separate dataframe for analysis:

* One hot encoding
* Grouping by neighbourhood
* Calculating the frequency of each venue

In [54]:
toronto_onehot = toronto_venues.loc[:, ['Neighbourhood', 'Venue Category']]

#1
# toronto_dummies = pd.get_dummies(toronto_onehot, columns=['Venue Category'], prefix='', prefix_sep='')

#2 alt
unique_venues = set(toronto_onehot['Venue Category'])
for venue in sorted(unique_venues):
    toronto_onehot[venue] = (toronto_onehot['Venue Category'] == venue).astype(float)

toronto_onehot.drop('Venue Category', axis=1, inplace=True)
toronto_onehot = toronto_onehot.groupby('Neighbourhood').mean()

print(toronto_onehot.shape)
toronto_onehot

(36, 239)


,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
Neighbourhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Berczy Park,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.030000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.0

In [55]:
pd.set_option('display.max_columns', 500)
toronto_onehot.describe().max(axis=1)

count    36.000000
mean      0.092277
std       0.083916
min       0.000000
25%       0.047078
50%       0.081717
75%       0.116711
max       0.500000
dtype: float64

In [56]:
clusters = KMeans(n_clusters=6, n_init=200).fit(toronto_onehot).labels_
clusters

array([0, 3, 3, 0, 3, 0, 0, 3, 2, 3, 0, 5, 0, 0, 3, 3, 3, 0, 3, 0, 0, 0,
       0, 0, 0, 3, 1, 3, 0, 3, 0, 4, 3, 3, 0, 3], dtype=int32)

In [57]:
def top_10_frequencies(row):
    
    row.sort_values(ascending=False, inplace=True)
    return row[0:10].index.values

toronto_most_common_venue = toronto_onehot.aggregate(top_10_frequencies, axis=1).to_frame()
toronto_most_common_venue = pd.DataFrame(toronto_most_common_venue.loc[:,0].to_list(), index=toronto_most_common_venue.index,
                                        columns=[f"Most common venue #{i+1}" for i in range(10)])
toronto_most_common_venue.insert(0, 'Label', clusters)

# Join and drop neighbourhoods that did not return any venues (3).
toronto_merged = toronto_neighbourhoods.join(toronto_most_common_venue, on='Neighbourhood').dropna(axis=0).reset_index(drop=True)
print(toronto_most_common_venue.shape, toronto_merged.shape)

toronto_merged

(36, 11) (36, 16)


,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Label,Most common venue #1,Most common venue #2,Most common venue #3,Most common venue #4,Most common venue #5,Most common venue #6,Most common venue #7,Most common venue #8,Most common venue #9,Most common venue #10
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655120,-79.362640,0.0,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center,Pub
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.660673,-79.390835,0.0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Restaurant,Bookstore,Juice Bar,Spa,Smoothie Shop,Burrito Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657390,-79.378040,0.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Diner,Chinese Restaurant,Fast Food Restaurant,Tanning Salon,Theater
3,M5C,Downtown Toronto,St. James Town,43.671100,-79.373590,0.0,Coffee Shop,Grocery Store,Bistro,Caribbean Restaurant,Sandwich Place,Pizza Place,Pharmacy,Convenience Store,Diner,Metro Station
4,M4E,East Toronto,The Beaches,43.674130,-79.296440,3.0,Japanese Restaurant,Bar,Grocery Store,Thai Restaurant,Burger Joint,Café,Sandwich Place,Breakfast Spot,Chocolate Shop,Juice Bar
5,M5E,Downtown Toronto,Berczy Park,43.648110,-79.375170,0.0,Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,Japanese Restaurant,Seafood Restaurant,Breakfast Spot,Gym,Gastropub
6,M5G,Downtown Toronto,Central Bay Street,43.655677,-79.383737,0.0,Coffee Shop,Clothing Store,Hotel,Bubble Tea Shop,Theater,Bookstore,Electronics Store,Café,Plaza,Diner
7,M6G,Downtown Toronto,Christie,43.673059,-79.422094,3.0,Grocery Store,Park,Coffee Shop,Beer Store,Italian Restaurant,American Restaurant,Gym,Nightclub,Café,Candy Store
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.649700,-79.382580,0.0,Coffee Shop,Hotel,Café,Restaurant,Gym,Steakhouse,American Restaurant,Japanese Restaurant,Salad Place,Asian Restaurant
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.663974,-79.430364,3.0,Bar,Café,Ethiopian Restaurant,Pub,Restaurant,Coffee Shop,Comedy Club,Bakery,Vegetarian / Vegan Restaurant,Park


In [58]:
address = 'Toronto, CA'
coords = geocoder.arcgis(address).json
latitude = coords['lat']
longitude = coords['lng']

In [59]:
Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

colors = ['#{:02X}{:02X}{:02X}'.format(*np.random.randint(0, 256, 3)) for label in set(clusters)]
    
def draw(row):
    
    label=folium.Popup(row.Neighbourhood + ' Cluster #' + str(int(row.Label)), parse_html=True)
    
    folium.Marker(
        [row['Neighbourhood Latitude'], row['Neighbourhood Longitude']],
        popup=label,
        tooltip="Cluster " + str(int(row.Label)),
        color=colors[int(row.Label)],
        icon=folium.Icon(icon='home', icon_color=colors[int(row.Label)]),
        fill=True
    ).add_to(Toronto)
        
toronto_merged.apply(draw, axis=1)

Toronto

# 0. Cluster 0 appears to include areas with increased Cafe activies and accommodation services.


In [60]:
first_cluster = toronto_merged[toronto_merged.Label == 0]
first_cluster.describe(include='all')[0:4]


,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Label,Most common venue #1,Most common venue #2,Most common venue #3,Most common venue #4,Most common venue #5,Most common venue #6,Most common venue #7,Most common venue #8,Most common venue #9,Most common venue #10
count,17,17,17,17.0,17.0,17.0,17,17,17,17,17,17,17,17,17,17
unique,17,3,17,NaN,NaN,NaN,2,10,9,11,12,14,15,14,16,13
top,M5G,Downtown Toronto,"Parkdale, Roncesvalles",NaN,NaN,NaN,Coffee Shop,Hotel,Café,Café,Japanese Restaurant,Japanese Restaurant,Thai Restaurant,Spa,Pub,Sandwich Place
freq,1,13,1,NaN,NaN,NaN,16,5,5,4,3,2,2,3,2,2


# 1. Cluster 1 includes only 1 outlier with seemingly more open landscape and more outdoor activities.
*Indeed, observing the map, you can see it is located in the vicinity of a stadium and a large open park.*

In [61]:
second_cluster = toronto_merged[toronto_merged.Label == 1]
second_cluster.describe(include='all')[0:4]

,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Label,Most common venue #1,Most common venue #2,Most common venue #3,Most common venue #4,Most common venue #5,Most common venue #6,Most common venue #7,Most common venue #8,Most common venue #9,Most common venue #10
count,1,1,1,1.0,1.0,1.0,1,1,1,1,1,1,1,1,1,1
unique,1,1,1,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1
top,M5N,Central Toronto,Roselawn,NaN,NaN,NaN,Garden,Pharmacy,Café,Skating Rink,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
freq,1,1,1,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1


# 2. Cluster 2 only includes one outlier with the Post Office as #1 Most common venue.



In [62]:
third_cluster = toronto_merged[toronto_merged.Label == 2]
third_cluster.describe(include='all')[0:4]

,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Label,Most common venue #1,Most common venue #2,Most common venue #3,Most common venue #4,Most common venue #5,Most common venue #6,Most common venue #7,Most common venue #8,Most common venue #9,Most common venue #10
count,1,1,1,1.0,1.0,1.0,1,1,1,1,1,1,1,1,1,1
unique,1,1,1,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1
top,M4P,Central Toronto,Davisville North,NaN,NaN,NaN,Post Office,Italian Restaurant,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
freq,1,1,1,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1


# 3. Cluster 3 appears to include more Bar/Coffee/Fastfood activity oriented neighbourhoods.


In [63]:
fourth_cluster = toronto_merged[toronto_merged.Label == 3]
fourth_cluster.describe(include='all')[0:4]

,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Label,Most common venue #1,Most common venue #2,Most common venue #3,Most common venue #4,Most common venue #5,Most common venue #6,Most common venue #7,Most common venue #8,Most common venue #9,Most common venue #10
count,15,15,15,15.0,15.0,15.0,15,15,15,15,15,15,15,15,15,15
unique,15,4,15,NaN,NaN,NaN,8,10,9,13,11,12,13,14,12,13
top,M5T,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",NaN,NaN,NaN,Bar,Restaurant,Café,Coffee Shop,Burger Joint,Café,Sandwich Place,Breakfast Spot,Chocolate Shop,Park
freq,1,6,1,NaN,NaN,NaN,3,3,4,2,3,2,3,2,2,2


# 4. Cluster 4 is again a single outlier, next to a local tram station. Appears to have less activities available.

In [64]:
fifth_cluster = toronto_merged[toronto_merged.Label == 4]
fifth_cluster.describe(include='all')[0:4]

,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Label,Most common venue #1,Most common venue #2,Most common venue #3,Most common venue #4,Most common venue #5,Most common venue #6,Most common venue #7,Most common venue #8,Most common venue #9,Most common venue #10
count,1,1,1,1.0,1.0,1.0,1,1,1,1,1,1,1,1,1,1
unique,1,1,1,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1
top,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",NaN,NaN,NaN,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Yoga Studio,Dumpling Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm
freq,1,1,1,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1


# 5. The final cluster, an outlier, appears to be a richer neighborhood with more retail and fitness activities than the rest outliers.

In [65]:
sixth_cluster = toronto_merged[toronto_merged.Label == 5]
sixth_cluster.describe(include='all')[0:4]

,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Label,Most common venue #1,Most common venue #2,Most common venue #3,Most common venue #4,Most common venue #5,Most common venue #6,Most common venue #7,Most common venue #8,Most common venue #9,Most common venue #10
count,1,1,1,1.0,1.0,1.0,1,1,1,1,1,1,1,1,1,1
unique,1,1,1,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1
top,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",NaN,NaN,NaN,Jewelry Store,Trail,Gym / Fitness Center,Sushi Restaurant,Park,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
freq,1,1,1,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1


Overall, fewer clusters could have been chosen.